In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

In [3]:
import json

f_raw = "../data/human_data_raw.jsonl"

with open(f_raw) as fin:
    data = [json.loads(line) for line in fin]
print(len(data))

1317


In [4]:
outputs = []
err_cnt_dist = dict()
err_type_dist = dict()
for idx,line in enumerate(data):
    # print(idx)
    
    flag = False
    if "CLASSIFICATION_JOB_2" in line["latestLabel"]["jsonResponse"]:
        flags = line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"]
        for i in range(len(flags)):
            if line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"][i]["name"] == "ERRORS_IN_THE_CORRECT_OUTPUT":
                flag = True
            elif line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_2"]["categories"][i]["name"] == "THE_CONTEXT_IS_VERY_COMPLEX_IM_NOT_SURE_OF_MY_ANSWER":
                flag = True
    if flag:
        continue

    labels = line["latestLabel"]["jsonResponse"]["CLASSIFICATION_JOB_0"]["categories"]
    feedback = []
    error_types = []
    for label in labels:
        feedback.extend([i["text"] for i in label["children"].values()])
        error_types.append(label["categoryName"])
        
    # priorities
    priorities = ['Coherence and Deduction', 'Veracity', 'Arithmetic', 'Commonsense'] #'Consistency with Context', 'Redundancy']
    
    if len(feedback) > 1:
        new_feedback = []
        new_error_type = []
        
        for priority in priorities:
            for type_idx in range(len(feedback)):
                if error_types[type_idx] == priority:
                    new_error_type.append(priority)
                    new_feedback.append(feedback[type_idx])
        
        feedback = new_feedback
        error_types = new_error_type
        
    if len(feedback) < 1: continue
                    
    
    # remove candidate
    for item_id in range(len(feedback)):
        if "victory is an election result in which the victorious candidate" in feedback[item_id]: 
            feedback[item_id] = feedback[item_id].replace("Candidates", "The answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("The candidate output", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate output'answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate'answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("the candidate had the answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidates answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate does not answer the question", "this does not answer the question")
        
        feedback[item_id] = feedback[item_id].replace("The candidates", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("The candidate's answer", "The answer")
        
        feedback[item_id] = feedback[item_id].replace("The output candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("the output candidate", "the answer")
        
        feedback[item_id] = feedback[item_id].replace("candidate's answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("the candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("candidate output", "the answer")
        feedback[item_id] = feedback[item_id].replace("Candidate output", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidate Output", "The answer")
        feedback[item_id] = feedback[item_id].replace("candidate answers", "it answers")
        feedback[item_id] = feedback[item_id].replace("Th candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Output Candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Output candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidate", "The answer")
        feedback[item_id] = feedback[item_id].replace("Candidates", "The answer")
        
        feedback[item_id] = feedback[item_id].replace("candidate", "the answer")
        
        
        feedback[item_id] = feedback[item_id].replace("The answer answers ", "The answer says ")
        feedback[item_id] = feedback[item_id].replace("the answer answers ", "the answer says ")
        feedback[item_id] = feedback[item_id].replace("The answer answered ", "The answer sayed ")
        feedback[item_id] = feedback[item_id].replace("the answer answered ", "the answer sayed ")
        feedback[item_id] = feedback[item_id].replace("The answer answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("The answer in the answer", "The answer")
        feedback[item_id] = feedback[item_id].replace("the answers' answer", "the answer")
        feedback[item_id] = feedback[item_id].replace("The answer's answer", "The answer")
        
        
        answer_loc_1 = feedback[item_id].find("answer")
        answer_loc_2 = feedback[item_id][answer_loc_1+5:].find("answer")
        if answer_loc_2 > -1 and answer_loc_2 < 10:
            print(feedback[item_id])
            print(" ")
    
    my_key = "candidate"
    for item in feedback:
        loc = -1
        if my_key in item:
            loc = item.find(my_key)
        
        if loc > -1:
            print(item[loc:loc+155])
    
    
        
    if len(feedback) in err_cnt_dist:
        err_cnt_dist[len(feedback)] += 1
    else:
        err_cnt_dist[len(feedback)] = 1
        
    for err_type in error_types:
        if err_type in err_type_dist:
            err_type_dist[err_type] += 1
        else:
            err_type_dist[err_type] = 1
        
    question = line["metadata"]["context"]
    answer = line["metadata"]["output_candidate"]
    
    critique = " ".join(feedback)
    if "correct output" in critique or "Correct output" in critique or "Correct Output" in critique:
#         print("prompt_id:", line["metadata"]["prompt_id"])
        continue
    
    outputs.append({"question": question,
                  "answer": answer,
                  "feedback": feedback,
                  "error_types": error_types})
print(len(outputs))
print(err_cnt_dist)
print(err_type_dist)

The answer never answers the question and focuses on a different conclusion. The hypothesis in the context says "the rabbit is big" but the answer is unknown because there is not enough information to determine that.
 
The steps given do not give the right answer, The answer divided the The blue cars speed with the Green cars speed instead of multiply to get the actual speed. From the context The blue car travels at a speed of 80m/h so the green travels at 80*8=640m/h. the the  red one is twice as green so it will be 640*2=1280m/h.
 
In their answer, the answer states that "they sold 6*3=18 shoes" but this number is actually the revenue from the shoes, not the number of shoes. The answer then proceeds to multiply this by 3 again which yields the whole calculation incorrect. The correct process should be to multiply the pairs of shoes by 3 to get $18, then multiply 18 shirts by 2 to get $36, and sum this to get the total of $54. If they divide their total earning equally, each will get 

In [5]:
version = 0

processed_outputs = []
err_type_dist = dict()

bad_count_redundancy = 10
bad_count_consistent = 10

for idx, output in enumerate(outputs):
    
    if len(output["feedback"]) == 1:
        if output["error_types"][0] == "No error":
            critique = output["feedback"][0]
        elif " correct" in output["feedback"][0]:
            critique = output["feedback"][0]
        elif output["error_types"][0] in ["Redundancy", "Consistency with Context"]:
            critique = output["feedback"][0]
        else:
            critique = "" + output["feedback"][0]
            
        if output["error_types"][0] == "Redundancy":
            if bad_count_redundancy > 0:
                bad_count_redundancy -= 1
            else:
                continue
        if output["error_types"][0] == "Consistency with Context":
            if bad_count_consistent > 0:
                bad_count_consistent -= 1
            else:
                continue
            
    elif len(output["feedback"]) == 2:
        critique = "Firstly, " + output["feedback"][0] + " Secondly, " + output["feedback"][1]
    elif len(output["feedback"]) == 3:
        critique = "Firstly, " + output["feedback"][0] + " Secondly, " + output["feedback"][1] + \
        " Thirdly, " + output["feedback"][2]
    
    if "in which the victorious candidate or party wins by an overwhelming margin. Therefore, the vote could not" in critique: continue
    
    for err_type in output["error_types"]:
        if err_type in err_type_dist:
            err_type_dist[err_type] += 1
        else:
            err_type_dist[err_type] = 1
    
#     if "Veracity" in output["error_types"]:
#         print(critique)
    if idx % 70 == 0:
        print(critique)
        print(output["error_types"])
        print()
    processed_outputs.append({"id":idx,"question":output["question"],"answer":output["answer"],
                "feedback":critique,"meta":{"domain":"human annotation"}})
    



print(len(processed_outputs))

with open(f"./v{version}.jsonl", "w") as fout:
    for output in processed_outputs:
        fout.write(f"{json.dumps(output)}\n")

Firstly, The math is off in the previous to last sentence, since the ages of the children should be subtracted, not summed. 15 - 1 - 4 -2 = 8 is the right answer. Secondly, "So the fourth child is 15 + 1 + 4 + 2 = 22 years old. The answer is 22." The fourth born child cannot be older than the first born child.  
['Coherence and Deduction', 'Commonsense']

Blowing on the open end won't help. After tearing a hole at the end of the paper, the rest can be pulled from the other side to release the straw.
['Commonsense']

After giving an explanation, the answer does not provide an answer.
['Coherence and Deduction']

The answer doesn't answer the question directly and there are missing steps. The hypothesis is not correct because if something eats the cat and it does not visit the cat then the cat needs the cow. The answer doesn't list any of the correct steps to get to the cat needing the cow.
['Coherence and Deduction']

2nd statement that Dave is not furry by the answer is incorrect. We d

In [6]:
print(err_type_dist)

{'Coherence and Deduction': 875, 'Commonsense': 226, 'Arithmetic': 161, 'Veracity': 143}
